<h1 align = center><b>Booth's Algorithm <br>for Signed Binary Multiplication</b></h1>

<h3>Importing necessary modules</h3>

In [27]:
from prettytable import PrettyTable

<h3>Taking Input</h3>

In [28]:
def take_input():
    mcand = int(input("Enter the Multiplicand: "))
    mlier = int(input("Enter the Multiplier: "))

    return mcand, mlier

<h3>Decimal to Binary Converter</h3>

In [29]:
def to_binary(*dec_number):
    bin_master_list = []
    def mechanism(dec_number):
        if(dec_number >= 1):
            mechanism(dec_number // 2)
            bin_num.append(dec_number % 2)
        
    for i in dec_number:
        bin_num = []
        mechanism(i)
        bin_master_list.append(bin_num)

    return bin_master_list   

<h3>Dynamic Bit Adjuster</h3>

In [30]:
def dynamic_bit_adjuster(list1, list2):
    l1 = len(list1)
    l2 = len(list2)

    if(l1 != l2):
        if(l1 > l2):
            list1.insert(0, 0)
            for i in range(l2, l1 + 1):
                list2.insert(0, 0)
        
        elif(l2 > l1):
            list2.insert(0, 0)
            for i in range(l1, l2 + 1):
                list1.insert(0, 0)
    else:
        list1.insert(0, 0)
        list2.insert(0, 0)

<h3>One's Complementer</h3>

In [31]:
def ones_complement(binary_num_list):
    for i in range(0, len(binary_num_list)):
        if binary_num_list[i] == 0:
            binary_num_list[i] = 1
        else:
            binary_num_list[i] = 0

<h3>Binary Additioner</h3>

In [32]:
def binary_add(list1, list2):
    carry = 0

    for i in range(len(list1) - 1, -1, -1): #reverse traversal
        if(carry == 0):
            if(list1[i] == 0 and list2[i] == 0):
                list1[i] = 0
            elif(list1[i] == 0 and list2[i] == 1):
                list1[i] = 1
            elif(list1[i] == 1 and list2[i] == 0):
                list1[i] = 1
            elif(list1[i] == 1 and list2[i] == 1):
                list1[i] = 0
                carry = 1   #! Quite important
                
        elif(carry == 1):
            if(list1[i] == 0 and list2[i] == 0):
                list1[i] = 1
                carry = 0

            elif(list1[i] == 0 and list2[i] == 1):
                list1[i] = 0
                carry = 1

            elif(list1[i] == 1 and list2[i] == 0):
                list1[i] = 0
                carry = 1

            elif(list1[1] == 1 and list2[i] == 1):
                list1[i] = 1
                carry = 1 #! Quite important

<h3>Two's Complementer</h3>

In [33]:
def twos_complement(list1):
    one = [1]
    for i in range(0, len(list1)-1):
        one.insert(0, 0)
    
    ones_complement(list1)
    binary_add(list1, one)

<h3>Arithmetic Right Shifter</h3>

In [34]:
#this will function properly only when both lists have equal lengths
def ar_right_shift(list1, list2):
    temp_bit1 = list1[-1]
    temp_bit2 = list2[-1] #last bits
    temp_copy1 = list1.copy()
    temp_copy2 = list2.copy()

    for i in range(1, len(list1)):
        list1[i] = temp_copy1[i - 1]
        list2[i] = temp_copy2[i - 1]
    list2[0] = temp_bit1

    return temp_bit2

<h3>List to String Converter</h3>

In [35]:
def list_to_string(list1):
    string = ""
    for i in list1:
        string += str(i)
    
    return string

<h3>Binary to Decimal Converter</h3>

In [36]:
def to_decimal(list1, list2, dec_mcand, dec_mlier):
    list3 = list1 + list2
    expo = 0
    summation = 0
    
    if((dec_mcand < 0 and dec_mlier > 0) or (dec_mlier < 0 and dec_mcand > 0)):
        twos_complement(list3)
        

    for i in range(len(list3)-1, -1, -1): #reverse traversal
        if(list3[i] == 1):
            summation += 2**(expo)
        expo += 1
    
    return summation

<h3>Main 'Booth's Algorithm' Function</h3>

In [37]:
def Booths_Algorithm():
    #taking input
    dec_mcand, dec_mlier = take_input()
    
    #check for input 0, program does work without this but better to have it not waste computation time if these are the inputs
    if dec_mcand == 0 or dec_mlier == 0:
        print("Final Answer:\n{} x {} = {}".format(dec_mcand, dec_mlier, 0))
        return

    #check for input 1 
    if dec_mcand == 1:
        print("Final Answer:\n{} x {} = {}".format(dec_mcand, dec_mlier, dec_mlier))
        return
    elif dec_mlier == 1:
        print("Final Answer:\n{} x {} = {}".format(dec_mcand, dec_mlier, dec_mcand))
        return

    #coversion from decimal to binary for both inputs
    bin_nums = to_binary(abs(dec_mcand), abs(dec_mlier))
    bin_mcand = bin_nums[0]  
    bin_mlier = bin_nums[1] 
    
    #adjusting bits for both multiplicand and multiplier
    dynamic_bit_adjuster(bin_mcand, bin_mlier)

    #taking two's complement if negative
    if(dec_mcand < 0):
        twos_complement(bin_mcand)
    if(dec_mlier < 0):
        twos_complement(bin_mlier)
    
    #preparation for the main algorithm
    A = [0 for i in range (0, len(bin_mcand))]
    Q = bin_mlier.copy()
    M = bin_mcand.copy()
    twos_complement(bin_mcand)      #this WILL change bin_mcand because of pass by reference
    m_M = bin_mcand.copy()
    Qm1 = 0                         #Q_minus_1 bit
    count = len(bin_mcand)
    table = PrettyTable(["A", "Q", "Q_-1", 'M', "Count", "Operation"])
    final_table = [[list_to_string(A), list_to_string(Q), Qm1, list_to_string(M), count, "Start"]]
    
    #logic/steps of the algorithm
    while(count > 0):
        if(Q[-1] == 1 and Qm1 == 0):
            binary_add(A, m_M)
            final_table.append([list_to_string(A), list_to_string(Q), Qm1, list_to_string(M), count, "A = A - M"]) #have to convert to string to mitigate reference assignment of modified list

        elif(Q[-1] == 0 and Qm1 == 1):
            binary_add(A, M)
            final_table.append([list_to_string(A), list_to_string(Q), Qm1, list_to_string(M), count, "A = A + M"])

        Qm1 = ar_right_shift(A, Q)
        final_table.append([list_to_string(A), list_to_string(Q), Qm1, list_to_string(M), count, "Arithmetic Right Shift"])
        count -= 1
    
    #printing the Operations table
    final_table.append(['', '', '', '', '', "Finish"])
    table.add_rows(final_table)
    print(table)

    #printing the final answer
    f_a = to_decimal(A, Q, dec_mcand, dec_mlier)
    if((dec_mcand < 0 and dec_mlier > 0) or (dec_mlier < 0 and dec_mcand > 0)): 
        print("\nFinal Answer:\n{} x {} = -{}".format(dec_mcand, dec_mlier, f_a))
    else:
        print("\nFinal Answer:\n{} x {} = {}".format(dec_mcand, dec_mlier, f_a))

<h3>Callers</h3>

In [38]:
Booths_Algorithm()

+--------+--------+------+--------+-------+------------------------+
|   A    |   Q    | Q_-1 |   M    | Count |       Operation        |
+--------+--------+------+--------+-------+------------------------+
| 000000 | 111011 |  0   | 010001 |   6   |         Start          |
| 101111 | 111011 |  0   | 010001 |   6   |       A = A - M        |
| 110111 | 111101 |  1   | 010001 |   6   | Arithmetic Right Shift |
| 111011 | 111110 |  1   | 010001 |   5   | Arithmetic Right Shift |
| 001100 | 111110 |  1   | 010001 |   4   |       A = A + M        |
| 000110 | 011111 |  0   | 010001 |   4   | Arithmetic Right Shift |
| 110101 | 011111 |  0   | 010001 |   3   |       A = A - M        |
| 111010 | 101111 |  1   | 010001 |   3   | Arithmetic Right Shift |
| 111101 | 010111 |  1   | 010001 |   2   | Arithmetic Right Shift |
| 111110 | 101011 |  1   | 010001 |   1   | Arithmetic Right Shift |
|        |        |      |        |       |         Finish         |
+--------+--------+------+--------

In [39]:
Booths_Algorithm()

+--------+--------+------+--------+-------+------------------------+
|   A    |   Q    | Q_-1 |   M    | Count |       Operation        |
+--------+--------+------+--------+-------+------------------------+
| 000000 | 010001 |  0   | 111011 |   6   |         Start          |
| 000101 | 010001 |  0   | 111011 |   6   |       A = A - M        |
| 000010 | 101000 |  1   | 111011 |   6   | Arithmetic Right Shift |
| 111101 | 101000 |  1   | 111011 |   5   |       A = A + M        |
| 111110 | 110100 |  0   | 111011 |   5   | Arithmetic Right Shift |
| 111111 | 011010 |  0   | 111011 |   4   | Arithmetic Right Shift |
| 111111 | 101101 |  0   | 111011 |   3   | Arithmetic Right Shift |
| 000100 | 101101 |  0   | 111011 |   2   |       A = A - M        |
| 000010 | 010110 |  1   | 111011 |   2   | Arithmetic Right Shift |
| 111101 | 010110 |  1   | 111011 |   1   |       A = A + M        |
| 111110 | 101011 |  0   | 111011 |   1   | Arithmetic Right Shift |
|        |        |      |        

In [40]:
Booths_Algorithm()

+--------+--------+------+--------+-------+------------------------+
|   A    |   Q    | Q_-1 |   M    | Count |       Operation        |
+--------+--------+------+--------+-------+------------------------+
| 000000 | 010001 |  0   | 000101 |   6   |         Start          |
| 111011 | 010001 |  0   | 000101 |   6   |       A = A - M        |
| 111101 | 101000 |  1   | 000101 |   6   | Arithmetic Right Shift |
| 000010 | 101000 |  1   | 000101 |   5   |       A = A + M        |
| 000001 | 010100 |  0   | 000101 |   5   | Arithmetic Right Shift |
| 000000 | 101010 |  0   | 000101 |   4   | Arithmetic Right Shift |
| 000000 | 010101 |  0   | 000101 |   3   | Arithmetic Right Shift |
| 111011 | 010101 |  0   | 000101 |   2   |       A = A - M        |
| 111101 | 101010 |  1   | 000101 |   2   | Arithmetic Right Shift |
| 000010 | 101010 |  1   | 000101 |   1   |       A = A + M        |
| 000001 | 010101 |  0   | 000101 |   1   | Arithmetic Right Shift |
|        |        |      |        

In [41]:
Booths_Algorithm()

+--------+--------+------+--------+-------+------------------------+
|   A    |   Q    | Q_-1 |   M    | Count |       Operation        |
+--------+--------+------+--------+-------+------------------------+
| 000000 | 000101 |  0   | 010001 |   6   |         Start          |
| 101111 | 000101 |  0   | 010001 |   6   |       A = A - M        |
| 110111 | 100010 |  1   | 010001 |   6   | Arithmetic Right Shift |
| 001000 | 100010 |  1   | 010001 |   5   |       A = A + M        |
| 000100 | 010001 |  0   | 010001 |   5   | Arithmetic Right Shift |
| 110011 | 010001 |  0   | 010001 |   4   |       A = A - M        |
| 111001 | 101000 |  1   | 010001 |   4   | Arithmetic Right Shift |
| 001010 | 101000 |  1   | 010001 |   3   |       A = A + M        |
| 000101 | 010100 |  0   | 010001 |   3   | Arithmetic Right Shift |
| 000010 | 101010 |  0   | 010001 |   2   | Arithmetic Right Shift |
| 000001 | 010101 |  0   | 010001 |   1   | Arithmetic Right Shift |
|        |        |      |        

In [42]:
Booths_Algorithm()

Final Answer:
0 x 9 = 0
